# TFace

> Face Image Quality model from [SDD-FIQA](https://github.com/Tencent/TFace).

In [ ]:
#| default_exp tface

In [ ]:
#| hide

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| exporti

# net = network()




In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()